In [6]:
import numpy as np
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Libraries imported


In [33]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [34]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
print(soup.find('table'))

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East Y

In [35]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields),3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_toronto= pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_toronto.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [44]:
df_toronto1=df_toronto[df_toronto['Borough']=='Not assigned']
df_toronto.drop(df_toronto1.index,inplace=True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [45]:
df_toronto[df_toronto['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


In [48]:
df_torontofinal = df_toronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_torontofinal.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_torontofinal

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [49]:
df_toronto.shape

(103, 3)

Second Question  Note: I did in the same note book for next 2 questions 

In [50]:
df_geotoronto = pd.read_csv('http://cocl.us/Geospatial_data')
df_geotoronto.columns = ['Postcode', 'Latitude', 'Longitude']

In [43]:
df_torontomerge = pd.merge(df_torontofinal, df_geotoronto, on=['Postcode'], how='inner')
df_torontomerge.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Third question

In [57]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [60]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbor in zip(df_torontomerge['Latitude'], df_torontomerge['Longitude'], df_torontomerge['Borough'], df_torontomerge['Neighbourhood']):
    label = '{}, {}'.format(neighbor, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [61]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: FNIVIW2GETSNME2B3EHWNWCGCKVXBSHIM4DE1QFCGL3HTSPW
CLIENT_SECRET:3P2CB1KTQWPUGWA2A4IE1FTKUZZMLIJUNQDMAXA3JP5DMKUU


In [62]:
df_torontomerge.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [63]:
neighborhood_latitude = df_torontomerge.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_torontomerge.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_torontomerge.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            10)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df_torontomerge['Neighbourhood'],
                                   latitudes=df_torontomerge['Latitude'],
                                   longitudes=df_torontomerge['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [71]:
print(toronto_venues.shape)
toronto_venues.head()

(692, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [73]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
Business reply mail Processing Centre,10,10,10,10,10,10


In [75]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 179 uniques categories.


In [77]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,...,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
1,"Alderwood, Long Branch",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
3,Bayview Village,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
5,Berczy Park,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.1,0.000000,0.0,0.0,0.0,0.00
6,"Birch Cliff, Cliffside West",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
8,Business reply mail Processing Centre,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.1000,0.1,0.1,0.2,0.1,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00


In [79]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3             Breakfast Spot  0.25
4                      Motel  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2        Dance Studio   0.1
3                 Pub   0.1
4         Coffee Shop   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop   0.2
1                Bridal Shop   0.1
2              Deli / Bodega   0.1
3  Middle Eastern Restaurant   0.1
4                      Diner   0.1


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0               Restaurant

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [143]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Creperie,Dog Run,Event Space,Convenience Store,Electronics Store,Cosmetics Shop
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Skating Rink,Athletics & Sports,Dance Studio,Pharmacy,Gym,Cosmetics Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Middle Eastern Restaurant,Deli / Bodega,Bank,Sushi Restaurant,Ice Cream Shop,Restaurant,Diner,Bridal Shop,Department Store
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Juice Bar,Thai Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,Sushi Restaurant,Pub,Café,Comfort Food Restaurant,Coffee Shop


Clustering

In [144]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 1, 1, 1, 2, 1, 1, 4, 1], dtype=int32)

In [145]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_torontomerge

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')


toronto_merged.head(103) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Concert Hall,Farmers Market,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Bar,Moving Target,Women's Store,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Breakfast Spot,Electronics Store,Intersection,Medical Center,Bank,Rental Car Location,Mexican Restaurant,Distribution Center,Event Space,Eastern European Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Women's Store,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Bank,Bakery,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Gas Station,Caribbean Restaurant,Thai Restaurant,Distribution Center,Discount Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2.0,Convenience Store,Playground,Health & Beauty Service,Women's Store,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Convenience Store,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,2.0,Bus Line,Bakery,Soccer Field,Park,Intersection,Bus Station,Ice Cream Shop,Dog Run,Electronics Store,Eastern European Restaurant
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,2.0,Motel,American Restaurant,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,Skating Rink,General Entertainment,College Stadium,Café,Women's Store,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [148]:
toronto_merged=toronto_merged.dropna()

In [149]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Moving Target,Women's Store,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Women's Store,Diner,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
6,Scarborough,1.0,Convenience Store,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
9,Scarborough,1.0,Skating Rink,General Entertainment,College Stadium,Café,Women's Store,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
10,Scarborough,1.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Light Rail Station,Cosmetics Shop,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore
18,North York,1.0,Bakery,Shopping Mall,Movie Theater,Burger Joint,Clothing Store,Chocolate Shop,Toy / Game Store,Salon / Barbershop,Tea Room,Restaurant
19,North York,1.0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
22,North York,1.0,Grocery Store,Café,Coffee Shop,Plaza,Home Service,Movie Theater,Butcher,Arts & Crafts Store,Pharmacy,Pizza Place
24,North York,1.0,Grocery Store,Café,Coffee Shop,Plaza,Home Service,Movie Theater,Butcher,Arts & Crafts Store,Pharmacy,Pizza Place
26,North York,1.0,Japanese Restaurant,Discount Store,Coffee Shop,Caribbean Restaurant,Sporting Goods Shop,Restaurant,Clothing Store,Bike Shop,Athletics & Sports,Italian Restaurant
